In [ ]:
# Install dan Import Library
!pip install transformers torch xgboost --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 MB 3.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

In [ ]:
# Load Dataset
file_path = '/content/preprocessed_hoax_dataset.csv'
df = pd.read_csv(file_path)
print("Informasi Dataset:")
print(df.info())
print("\n5 Baris Pertama Dataset:")
print(df.head())

Informasi Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31678 entries, 0 to 31677
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   judul                 31678 non-null  object
 1   narasi                31678 non-null  object
 2   label                 31678 non-null  int64 
 3   judul_length          31678 non-null  int64 
 4   narasi_length         31678 non-null  int64 
 5   text_clean            31666 non-null  object
 6   text_casefoldingText  31666 non-null  object
 7   text_slangwords       31666 non-null  object
 8   text_tokenizingText   31678 non-null  object
 9   text_stopword         31678 non-null  object
 10  text_akhir            31660 non-null  object
dtypes: int64(3), object(8)
memory usage: 2.7+ MB
None

5 Baris Pertama Dataset:
                                               judul  \
0  Anies di Milad BKMT: Pengajian Menghasilkan Ib...   
1  Edy Soal Pilgub Sumut: Kal

In [ ]:
# Data Cleaning & Split Data
X = df['text_akhir'].astype(str)
y = df['label']
data_clean = pd.concat([X, y], axis=1).dropna(subset=['text_akhir'])
X_clean = data_clean['text_akhir']
y_clean = data_clean['label']

X_train, X_test, y_train, y_test = train_test_split(
    X_clean, y_clean,
    test_size=0.2,
    random_state=42,
    stratify=y_clean
)
print("✅ Data berhasil dibersihkan dan di-split.")

✅ Data berhasil dibersihkan dan di-split.


In [ ]:
# Load IndoBERT Tokenizer & Model
tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p1")
model = AutoModel.from_pretrained("indobenchmark/indobert-base-p1")
model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/229k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(50000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [ ]:
# Fungsi Ekstraksi Embedding IndoBERT
def get_bert_embeddings(texts, tokenizer, model, max_length=128, batch_size=32):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        inputs = tokenizer(
            batch_texts,
            padding=True,
            truncation=True,
            max_length=max_length,
            return_tensors="pt"
        )
        with torch.no_grad():
            outputs = model(**inputs)
            batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
            embeddings.append(batch_embeddings)
    return np.vstack(embeddings)

In [ ]:
# Ekstrak Embedding IndoBERT
print("Ekstraksi embedding IndoBERT untuk data train...")
X_train_embed = get_bert_embeddings(X_train.tolist(), tokenizer, model)
print("Ekstraksi embedding IndoBERT untuk data test...")
X_test_embed = get_bert_embeddings(X_test.tolist(), tokenizer, model)

Ekstraksi embedding IndoBERT untuk data train...
Ekstraksi embedding IndoBERT untuk data test...


In [ ]:
# Modeling & Evaluasi Multi Model
models = {
    "Logistic Regression": LogisticRegression(max_iter=200),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

results = []
best_auc = 0
best_model = None
best_model_name = ""

for name, clf in models.items():
    print(f"\nModel: {name}")
    clf.fit(X_train_embed, y_train)
    y_pred = clf.predict(X_test_embed)
    y_proba = clf.predict_proba(X_test_embed)[:,1]
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    roc_auc = roc_auc_score(y_test, y_proba)
    print(f"ROC-AUC: {roc_auc:.4f}")
    results.append({
        "Model": name,
        "ROC-AUC": roc_auc,
        "Akurasi": clf.score(X_test_embed, y_test)
    })
    # Simpan model terbaik
    if roc_auc > best_auc:
        best_auc = roc_auc
        best_model = clf
        best_model_name = name


Model: Logistic Regression
Confusion Matrix:
[[4013  329]
 [  60 1930]]
Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.92      0.95      4342
           1       0.85      0.97      0.91      1990

    accuracy                           0.94      6332
   macro avg       0.92      0.95      0.93      6332
weighted avg       0.94      0.94      0.94      6332

ROC-AUC: 0.9817
Distribusi Hasil Prediksi di Data Test:
0    4073
1    2259
Name: count, dtype: int64

Model: Random Forest
Confusion Matrix:
[[4011  331]
 [  58 1932]]
Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.92      0.95      4342
           1       0.85      0.97      0.91      1990

    accuracy                           0.94      6332
   macro avg       0.92      0.95      0.93      6332
weighted avg       0.94      0.94      0.94      6332

ROC-AUC: 0.9770
Distribusi Hasil Prediksi di Data Test:
0 

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [14:43:47] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



Model: XGBoost
Confusion Matrix:
[[4012  330]
 [  40 1950]]
Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.92      0.96      4342
           1       0.86      0.98      0.91      1990

    accuracy                           0.94      6332
   macro avg       0.92      0.95      0.93      6332
weighted avg       0.95      0.94      0.94      6332

ROC-AUC: 0.9834
Distribusi Hasil Prediksi di Data Test:
0    4052
1    2280
Name: count, dtype: int64

Model terbaik: XGBoost dengan ROC-AUC: 0.9834


In [ ]:
# Visualisasi Perbandingan Model\

results_df = pd.DataFrame(results)
plt.figure(figsize=(8,4))
sns.barplot(data=results_df, x="Model", y="ROC-AUC")
plt.title("Perbandingan ROC-AUC Tiap Model (Fitur IndoBERT)")
plt.ylim(0,1)
plt.show()
print("\nRingkasan Hasil:")
print(results_df)
print(f"\nModel terbaik: {best_model_name} (ROC-AUC: {best_auc:.4f})")

Distribusi Label Asli di Data Test:
label
0    4342
1    1990
Name: count, dtype: int64
Distribusi Hasil Prediksi di Data Test:
0    4052
1    2280
Name: count, dtype: int64


In [ ]:
# Prediksi Data Random (Contoh Baru)
contoh_berita = [
    "Pemerintah mengumumkan vaksin gratis untuk seluruh masyarakat.",
    "Hoaks! Minum air garam bisa menyembuhkan COVID-19.",
    "Banjir besar melanda Jakarta hari ini.",
    "Viral! Makan bawang putih bisa menyembuhkan kanker, benarkah?"
]
contoh_embed = get_bert_embeddings(contoh_berita, tokenizer, model)
prediksi = best_model.predict(contoh_embed)
for teks, label in zip(contoh_berita, prediksi):
    print(f"Teks: {teks}")
    print(f"Prediksi: {'Hoaks' if label == 1 else 'Bukan Hoaks'}\n")

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [14:50:58] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


✅ Model terbaik berhasil disimpan dengan nama 'model_hoax_filter.pkl'.
✅ Vectorizer TF-IDF berhasil disimpan dengan nama 'tfidf_vectorizer.pkl'.


In [ ]:
# Simpan Model Terbaik dan Tokenizer
joblib.dump(best_model, 'model_hoax_filter_indobert.pkl')
tokenizer.save_pretrained('./tokenizer_indobert/')
print("✅ Model terbaik dan tokenizer IndoBERT berhasil disimpan.")